<a href="https://colab.research.google.com/github/skywalker0803r/c620/blob/main/C620.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)
import numpy as np
import joblib
from tqdm import tqdm_notebook as tqdm

# X1
def get_Combined_Feed_Rate(df):
    idx = df.iloc[0,6:].values.tolist()
    start = 218
    temp = df.iloc[start:start+41].dropna(axis=1)
    cols = [ 'Combined_Feed_Rate_' + str(i).strip(' ') +'_'+ str(j).strip(' ') for i,j in zip(temp.iloc[:,0].values,temp.iloc[:,1].values)]
    values = temp.iloc[:,2:].T
    values.index = idx
    values.columns = cols
    return values.astype('float32')

# X2
def get_c670_Upper_Feed_Blending_Factor(df):
    idx = df.iloc[0,6:].values.tolist()
    start = 347
    temp = df.iloc[start:start+41].dropna(axis=1)
    cols = [ 'c670_Upper_Feed_Blending_Factor_' + str(i).strip(' ') +'_'+ str(j).strip(' ') for i,j in zip(temp.iloc[:,0].values,temp.iloc[:,1].values)]
    vals = temp.iloc[:,3:].T
    vals.index = idx
    vals.columns = cols
    return vals.astype('float32')

# X3
def get_c670_Lower_Feed_Blending_Factor(df):
    idx = df.iloc[0,6:].values.tolist()
    start = 389
    temp = df.iloc[start:start+41].dropna(axis=1)
    cols = [ 'c670_Lower_Feed_Blending_Factor_' + str(i).strip(' ') +'_'+ str(j).strip(' ') for i,j in zip(temp.iloc[:,0].values,temp.iloc[:,1].values)]
    vals = temp.iloc[:,3:].T
    vals.index = idx
    vals.columns = cols
    return vals.astype('float32')

# Y1
def get_c670_Dist_Split_Factor(df):
    idx = df.iloc[0,6:].values.tolist()
    start = 262
    temp = df.iloc[start:start+41].dropna(axis=1)
    cols = [ 'c670_Dist_' + str(i).strip(' ') +'_'+ str(j).strip(' ') for i,j in zip(temp.iloc[:,0].values,temp.iloc[:,1].values)]
    vals = temp.iloc[:,3:].T
    vals.index = idx
    vals.columns = cols
    return vals.astype('float32')

# Y2
def get_c670_Btms_Split_Factor(df):
    idx = df.iloc[0,6:].values.tolist()
    start = 304
    temp = df.iloc[start:start+41].dropna(axis=1)
    cols = [ 'c670_Btms_' + str(i).strip(' ') +'_'+ str(j).strip(' ') for i,j in zip(temp.iloc[:,0].values,temp.iloc[:,1].values)]
    vals = temp.iloc[:,3:].T
    vals.index = idx
    vals.columns = cols
    return vals.astype('float32')

# WT1
def get_c670_Dist_wt(df):
    idx = df.iloc[0,6:].values.tolist()
    start = 130
    temp = df.iloc[start:start+41].dropna(axis=1)
    cols = [ 'c670_Dist_' + str(i).strip(' ') +'_'+ str(j).strip(' ') for i,j in zip(temp.iloc[:,0].values,temp.iloc[:,1].values)]
    vals = temp.iloc[:,3:].T
    vals.index = idx
    vals.columns = cols
    return vals.astype('float32')

# WT2
def get_c670_Btms_wt(df):
    idx = df.iloc[0,6:].values.tolist()
    start = 174
    temp = df.iloc[start:start+41].dropna(axis=1)
    cols = [ 'c670_Btms_' + str(i).strip(' ') +'_'+ str(j).strip(' ') for i,j in zip(temp.iloc[:,0].values,temp.iloc[:,1].values)]
    vals = temp.iloc[:,3:].T
    vals.index = idx
    vals.columns = cols
    return vals.astype('float32')

# ETL
def ETL(filepath):
    train = pd.DataFrame()
    data = pd.read_excel(filepath,sheet_name = None)
    sheet_names = list(data.keys())
    for i in tqdm(sheet_names):
        df = data.get(i)
        # input
        f1 = get_Combined_Feed_Rate(df)
        b1 = get_c670_Upper_Feed_Blending_Factor(df)
        b2 = get_c670_Lower_Feed_Blending_Factor(df)
        # sf
        sf1 = get_c670_Dist_Split_Factor(df)
        sf2 = get_c670_Btms_Split_Factor(df)
        # output wt 
        wt1 = get_c670_Dist_wt(df)
        wt2 = get_c670_Btms_wt(df)
        rows = pd.concat([f1,b1,b2,sf1,sf2,wt1,wt2],axis=1)
        # append
        if len(train) == 0:
            train = rows
        else:
            train = train.append(rows)
        # print info
        print("ep:{} shape:{}".format(i,train.shape))
    return train

In [25]:
filename = 'Aroma-2 C670_Split Factors Calculation for AI Training_Overall_Data.xlsx'
cleaned_df = ETL(filename)

ep:001~012 shape:(1296, 287)
ep:013~024 shape:(2592, 287)
ep:025~036 shape:(3888, 287)
ep:037~048 shape:(5184, 287)
ep:049~060 shape:(6480, 287)
ep:061~072 shape:(7776, 287)
ep:073~084 shape:(9072, 287)
ep:085~096 shape:(10368, 287)
ep:097~108 shape:(11664, 287)
ep:109~120 shape:(12960, 287)
ep:121~132 shape:(14256, 287)
ep:133~144 shape:(15552, 287)



In [26]:
cleaned_df.head()

,Combined_Feed_Rate_Hydrogen_wt%,Combined_Feed_Rate_Methane_wt%,Combined_Feed_Rate_Ethane_wt%,Combined_Feed_Rate_Propane_wt%,Combined_Feed_Rate_n-Butane_wt%,Combined_Feed_Rate_n-Pentane_wt%,Combined_Feed_Rate_n-Hexane_wt%,Combined_Feed_Rate_Benzene_wt%,Combined_Feed_Rate_Cyclohexane_wt%,Combined_Feed_Rate_n-Heptane_wt%,Combined_Feed_Rate_Water_wt%,Combined_Feed_Rate_Methylcyclohexane_wt%,Combined_Feed_Rate_Toluene_wt%,Combined_Feed_Rate_n-Octane_wt%,Combined_Feed_Rate_n-Propylcyclopentane_wt%,Combined_Feed_Rate_Ethylcyclohexane_wt%,Combined_Feed_Rate_Ethylbenzene_wt%,Combined_Feed_Rate_p-Xylene_wt%,Combined_Feed_Rate_m-Xylene_wt%,Combined_Feed_Rate_o-Xylene_wt%,Combined_Feed_Rate_n-Nonane_wt%,Combined_Feed_Rate_i-Propylbenzene_wt%,Combined_Feed_Rate_n-Propylcyclohexane_wt%,Combined_Feed_Rate_n-Propylbenzene_wt%,Combined_Feed_Rate_1-Methyl-3-ethylbenzene_wt%,Combined_Feed_Rate_1-Methyl-4-ethylbenzene_wt%,"Combined_Feed_Rate_1,3,5-Trimethylbenzene_wt%",Combined_Feed_Rate_1-Methyl-2-ethylbenzene_wt%,"Combined_Feed_Rate_1,2,4-Trimethylbenzene_wt%",Combined_Feed_Rate_tert-Butylcyclohexane_wt%,"Combined_Feed_Rate_1,2,3-Trimethylbenzene_wt%",Combined_Feed_Rate_Indane_wt%,Combined_Feed_Rate_1-Methyl-4-n-propylbenzene_wt%,"Combined_Feed_Rate_1,2-Diethylbenzene_wt%",Combined_Feed_Rate_5-Ethyl-m-xylene_wt%,"Combined_Feed_Rate_1,4-Diethylbenzene_wt%","Combined_Feed_Rate_1,2,3,5-Tetramethylbenzene_wt%",Combined_Feed_Rate_n-Pentylbenzene_wt%,Combined_Feed_Rate_n-Hexylbenzene_wt%,Combined_Feed_Rate_Nitrogen_wt%,Combined_Feed_Rate_Oxygen_wt%,c670_Upper_Feed_Blending_Factor_Hydrogen_Fraction,c670_Upper_Feed_Blending_Factor_Methane_Fraction,c670_Upper_Feed_Blending_Factor_Ethane_Fraction,c670_Upper_Feed_Blending_Factor_Propane_Fraction,c670_Upper_Feed_Blending_Factor_n-Butane_Fraction,c670_Upper_Feed_Blending_Factor_n-Pentane_Fraction,c670_Upper_Feed_Blending_Factor_n-Hexane_Fraction,c670_Upper_Feed_Blending_Factor_Benzene_Fraction,c670_Upper_Feed_Blending_Factor_Cyclohexane_Fraction,...,c670_Dist_1-Methyl-4-n-propylbenzene_wt%,"c670_Dist_1,2-Diethylbenzene_wt%",c670_Dist_5-Ethyl-m-xylene_wt%,"c670_Dist_1,4-Diethylbenzene_wt%","c670_Dist_1,2,3,5-Tetramethylbenzene_wt%",c670_Dist_n-Pentylbenzene_wt%,c670_Dist_n-Hexylbenzene_wt%,c670_Dist_Nitrogen_wt%,c670_Dist_Oxygen_wt%,c670_Btms_Hydrogen_wt%,c670_Btms_Methane_wt%,c670_Btms_Ethane_wt%,c670_Btms_Propane_wt%,c670_Btms_n-Butane_wt%,c670_Btms_n-Pentane_wt%,c670_Btms_n-Hexane_wt%,c670_Btms_Benzene_wt%,c670_Btms_Cyclohexane_wt%,c670_Btms_n-Heptane_wt%,c670_Btms_Water_wt%,c670_Btms_Methylcyclohexane_wt%,c670_Btms_Toluene_wt%,c670_Btms_n-Octane_wt%,c670_Btms_n-Propylcyclopentane_wt%,c670_Btms_Ethylcyclohexane_wt%,c670_Btms_Ethylbenzene_wt%,c670_Btms_p-Xylene_wt%,c670_Btms_m-Xylene_wt%,c670_Btms_o-Xylene_wt%,c670_Btms_n-Nonane_wt%,c670_Btms_i-Propylbenzene_wt%,c670_Btms_n-Propylcyclohexane_wt%,c670_Btms_n-Propylbenzene_wt%,c670_Btms_1-Methyl-3-ethylbenzene_wt%,c670_Btms_1-Methyl-4-ethylbenzene_wt%,"c670_Btms_1,3,5-Trimethylbenzene_wt%",c670_Btms_1-Methyl-2-ethylbenzene_wt%,"c670_Btms_1,2,4-Trimethylbenzene_wt%",c670_Btms_tert-Butylcyclohexane_wt%,"c670_Btms_1,2,3-Trimethylbenzene_wt%",c670_Btms_Indane_wt%,c670_Btms_1-Methyl-4-n-propylbenzene_wt%,"c670_Btms_1,2-Diethylbenzene_wt%",c670_Btms_5-Ethyl-m-xylene_wt%,"c670_Btms_1,4-Diethylbenzene_wt%","c670_Btms_1,2,3,5-Tetramethylbenzene_wt%",c670_Btms_n-Pentylbenzene_wt%,c670_Btms_n-Hexylbenzene_wt%,c670_Btms_Nitrogen_wt%,c670_Btms_Oxygen_wt%
001-001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218932,0.000173,0.003677,0.0,0.014934,50.471397,0.019317,0.019702,0.017998,2.190489,8.061416,17.576529,7.120585,0.031811,0.005885,0.040147,0.006866,0.962607,0.504568,2.554874,0.283670,6.523591,0.002977,1.041622,0.034328,0.001568,0.022715,0.881749,0.0,1.014159,0.259915,0.111812,0.0,0.0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.002896,0.000159,...,1.385400e-21,2.605215e-22,1.690884e-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.679792e-21,1.092233e-09,2.874164e-12,6.766402e-10,0.0,3.254346

In [27]:
cleaned_df.to_csv('../cleaned/c670_train.csv')

In [4]:
import pandas as pd
df = pd.read_csv('/home/ubuntu/Desktop/c620/ETL/cleaned/c670_train.csv',index_col=0)

In [5]:
def get_col(df,name):
    return df.columns[df.columns.str.contains(name)].tolist()

In [6]:
df.head()

,Combined_Feed_Rate_Hydrogen_wt%,Combined_Feed_Rate_Methane_wt%,Combined_Feed_Rate_Ethane_wt%,Combined_Feed_Rate_Propane_wt%,Combined_Feed_Rate_n-Butane_wt%,Combined_Feed_Rate_n-Pentane_wt%,Combined_Feed_Rate_n-Hexane_wt%,Combined_Feed_Rate_Benzene_wt%,Combined_Feed_Rate_Cyclohexane_wt%,Combined_Feed_Rate_n-Heptane_wt%,...,c670_Btms_Indane_wt%,c670_Btms_1-Methyl-4-n-propylbenzene_wt%,"c670_Btms_1,2-Diethylbenzene_wt%",c670_Btms_5-Ethyl-m-xylene_wt%,"c670_Btms_1,4-Diethylbenzene_wt%","c670_Btms_1,2,3,5-Tetramethylbenzene_wt%",c670_Btms_n-Pentylbenzene_wt%,c670_Btms_n-Hexylbenzene_wt%,c670_Btms_Nitrogen_wt%,c670_Btms_Oxygen_wt%
001-001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218932,0.000173,0.003677,...,0.069671,0.003183,0.046102,1.789543,0.0,2.058273,0.527507,0.226928,0.0,0.0
001-002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218930,0.000173,0.003676,...,0.069671,0.003183,0.046102,1.789543,0.0,2.058273,0.527507,0.226928,0.0,0.0
001-003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218930,0.000173,0.003676,...,0.069671,0.003183,0.046102,1.789543,0.0,2.058273,0.527507,0.226928,0.0,0.0
001-004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218987,0.000173,0.003675,...,0.069671,0.003183,0.046102,1.789543,0.0,2.058273,0.527507,0.226928,0.0,0.0
001-005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.218986,0.000173,0.003674,...,0.069671,0.003183,0.046102,1.789543,0.0,2.058273,0.527507,0.226928,0.0,0.0


In [7]:
dataset = {}
dataset['c670_Combined_Feed'] = df[get_col(df,'Combined')]
dataset['c670_Upper_Feed_Blending_Factor'] = df[get_col(df,'c670_Upper_Feed_Blending_Factor')]
dataset['c670_Lower_Feed_Blending_Factor'] = df[get_col(df,'c670_Lower_Feed_Blending_Factor')]

dataset['c670_Dist_Factor'] = df[get_col(df,'Dist')[:41]]
dataset['c670_Dist_wt'] = df[get_col(df,'Dist')][41:]

dataset['c670_Btms_Factor'] = df[get_col(df,'Btms')[:41]]
dataset['c670_Btms_wt'] = df[get_col(df,'Btms')[41:]]

In [8]:
dataset.keys()

dict_keys(['c670_Combined_Feed', 'c670_Upper_Feed_Blending_Factor', 'c670_Lower_Feed_Blending_Factor', 'c670_Dist_Factor', 'c670_Dist_wt', 'c670_Btms_Factor', 'c670_Btms_wt'])

In [9]:
import joblib
joblib.dump(dataset,'../cleaned/c670_train_dict.pkl')

['../cleaned/c670_train_dict.pkl']